In [12]:
# Cell 1: Import libraries
import numpy as np
import laspy
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import rerun as rr
import open3d as o3d

In [13]:
def calculate_ndvi(red, nir):
    nir = np.array(nir, dtype=np.float64)
    red = np.array(red, dtype=np.float64)
    
    return (nir - red) / (nir + red + 1e-8)  # Add small epsilon to avoid division by zero
    # Cap the NDVI value between -1 and 1
    #ndvi_capped = max(-1, min(1, ndvi))
    #return ndvi_capped

In [14]:
# Files
input_file = "tud_sim/tudcampus_class1.laz"
output_file = "tud_sim/tudcampus_sim_w_ndvi_new.laz"
#processed_las = process_laz_file(input_file, output_file)

In [15]:
# Open LAS
with laspy.open(input_file) as infile:
    las = infile.read()

    print(list(las.header.point_format.dimension_names))

    red = las.red
    nir = las.nir


['X', 'Y', 'Z', 'intensity', 'return_number', 'number_of_returns', 'synthetic', 'key_point', 'withheld', 'overlap', 'scanner_channel', 'scan_direction_flag', 'edge_of_flight_line', 'classification', 'user_data', 'scan_angle', 'point_source_id', 'gps_time', 'red', 'green', 'blue', 'nir', 'Amplitude', 'Reflectance', 'Deviation']


In [16]:
# Calculate NDVI
ndvi_calc = calculate_ndvi(red, nir)

In [17]:
# Make the new point cloud
out_las = las
out_las.add_extra_dim(laspy.ExtraBytesParams(name="ndvi", type=np.float32))
out_las.ndvi = ndvi_calc

print(list(out_las.header.point_format.dimension_names))

['X', 'Y', 'Z', 'intensity', 'return_number', 'number_of_returns', 'synthetic', 'key_point', 'withheld', 'overlap', 'scanner_channel', 'scan_direction_flag', 'edge_of_flight_line', 'classification', 'user_data', 'scan_angle', 'point_source_id', 'gps_time', 'red', 'green', 'blue', 'nir', 'Amplitude', 'Reflectance', 'Deviation', 'ndvi']


In [18]:
# Do the filtering of the points
# Filter 1: Filter out points with NDVI value under 0.5
ndvi_threshold = -1
out_las = out_las[out_las.ndvi > ndvi_threshold]

# Filter 2: Remove points with single return
out_las = out_las[out_las.number_of_returns > 1]

# Filter 3: Remove last return of all remaining points
out_las = out_las[out_las.return_number != out_las.number_of_returns]

In [19]:
# Remove outliers using SOR
def remove_outliers(las_data, nb_neighbors=20, std_ratio=2.0):
    # Convert LAS data to Open3D point cloud
    xyz = np.vstack((las_data.x, las_data.y, las_data.z)).transpose()
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(xyz)

    # Perform statistical outlier removal
    cl, ind = pcd.remove_statistical_outlier(nb_neighbors=nb_neighbors, std_ratio=std_ratio)
    
    # Filter the original LAS data
    filtered_las = las_data[ind]
    
    return filtered_las

# After your previous filtering steps:
out_las = remove_outliers(out_las, 20, 2.0)

In [20]:
def visualize_point_cloud_with_rerun(las_data, sample_size=None):
    # Initialize rerun
    rr.init("LAS Point Cloud Viewer - NDVI, NIR, Red")

    # Check if red and nir bands exist
    if not (hasattr(las_data, 'red') and hasattr(las_data, 'nir')):
        raise ValueError("The LAS file does not contain 'red' and 'nir' bands required for NDVI calculation.")

    # Calculate NDVI
    ndvi = calculate_ndvi(las_data.red, las_data.nir)

    # Sample the data if sample_size is provided
    if sample_size is not None and sample_size < len(las_data.points):
        indices = np.random.choice(len(las_data.points), sample_size, replace=False)
        points = np.column_stack((las_data.x[indices], las_data.y[indices], las_data.z[indices]))
        ndvi_values = ndvi[indices]
        nir_values = las_data.nir[indices]
        red_values = las_data.red[indices]
    else:
        points = np.column_stack((las_data.x, las_data.y, las_data.z))
        ndvi_values = ndvi
        nir_values = las_data.nir
        red_values = las_data.red

    # Normalize NIR and Red values to 0-1 range
    nir_normalized = (nir_values - np.min(nir_values)) / (np.max(nir_values) - np.min(nir_values))
    red_normalized = (red_values - np.min(red_values)) / (np.max(red_values) - np.min(red_values))

    # Create color array: Red channel for Red values, Green for NDVI, Blue for NIR
    colors = np.column_stack((
        red_normalized,
        (ndvi_values + 1) / 2,  # NDVI is already in [-1, 1] range, normalize to [0, 1]
        nir_normalized
    ))

    # Log the point cloud to rerun
    rr.log("point_cloud", 
           rr.Points3D(
               positions=points,
               colors=colors,
               #radii=0.5,  # Adjust point size as needed
               labels=[f"NDVI: {ndvi:.3f}, NIR: {nir:.3f}, Red: {red:.3f}" 
                       for ndvi, nir, red in zip(ndvi_values, nir_values, red_values)]
           ))

    # Log some text to provide context
    rr.log("info", rr.TextLog("Point cloud: Red channel = Red band, Green channel = NDVI, Blue channel = NIR band. Hover for values."))

    print(f"Visualization ready. Run 'rerun' in terminal to view.")

visualize_point_cloud_with_rerun(out_las, sample_size=100000)

Visualization ready. Run 'rerun' in terminal to view.


In [21]:
rr.notebook_show()

Viewer()

In [22]:
# Write the point cloud to file
out_las.write(output_file)

# Continue with Open3D to calculate and output alpha shape